# Генерация тренировочного и тестового датасетов

# Скачиваем архив

In [2]:
!wget "https://courses.openedu.ru/assets/courseware/v1/59dbdef6c7b548953210145ec4ffdca3/asset-v1:ITMOUniversity+INFSECBAS+fall_2021_ITMO+type@asset+block/Task_1_prepprocessed.csv.zip"
!unzip Task_1_prepprocessed.csv

--2021-10-08 21:48:42--  https://courses.openedu.ru/assets/courseware/v1/59dbdef6c7b548953210145ec4ffdca3/asset-v1:ITMOUniversity+INFSECBAS+fall_2021_ITMO+type@asset+block/Task_1_prepprocessed.csv.zip
Resolving courses.openedu.ru (courses.openedu.ru)... 178.248.235.109
Connecting to courses.openedu.ru (courses.openedu.ru)|178.248.235.109|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://cdn.openedu.ru/ESZCBLZ/courseware/v1/59dbdef6c7b548953210145ec4ffdca3/asset-v1:ITMOUniversity+INFSECBAS+fall_2021_ITMO+type@asset+block/Task_1_prepprocessed.csv.zip [following]
--2021-10-08 21:48:42--  https://cdn.openedu.ru/ESZCBLZ/courseware/v1/59dbdef6c7b548953210145ec4ffdca3/asset-v1:ITMOUniversity+INFSECBAS+fall_2021_ITMO+type@asset+block/Task_1_prepprocessed.csv.zip
Resolving cdn.openedu.ru (cdn.openedu.ru)... 93.93.88.30
Connecting to cdn.openedu.ru (cdn.openedu.ru)|93.93.88.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Lengt

# Меняем по заданию

In [42]:
test_size = 0.3
random_state = 65
n_estimators = 10
n_jobs = 10

text_URL = "https://courses.openedu.ru/assets/courseware/v1/0a3b28bd6d49a6f3509c1e281dca1613/asset-v1:ITMOUniversity+INFSECBAS+fall_2021_ITMO+type@asset+block/task_1_P_test_2.csv"

# Код программы

In [92]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_fscore_support
import scipy as sp

df = pd.read_csv('Task_1_prepprocessed.csv')
df = df.dropna()
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, 5], df.iloc[:,0], test_size=test_size, random_state=random_state)

df_copy = df.copy(deep = True)

for i in range(len(df_copy)):
    if df.iat[i, 4] != '':
        try:
            df_copy.iat[i, 5] = df_copy.iat[i, 4] + ' ' + df_copy.iat[i, 5]
        except:
            print("Something got wrong!")
            
x_train_b, x_test_b, _, _ = train_test_split(df_copy.iloc[:, [1, 5]], df_copy.iloc[:,0], test_size=test_size, random_state=random_state)

vectorizer_a = TfidfVectorizer()
x_train_a = vectorizer_a.fit_transform(x_train)
x_test_a = vectorizer_a.transform(x_test)

vectorizer_b = TfidfVectorizer()
x_train_b = sp.sparse.hstack((vectorizer_b.fit_transform(x_train_b.iloc[:, 1]), x_train_b.iloc[:, 0].values.reshape(len(x_train_b.iloc[:, 0]),1)))
x_test_b = sp.sparse.hstack((vectorizer_b.transform(x_test_b.iloc[:, 1]), x_test_b.iloc[:, 0].values.reshape(len(x_test_b.iloc[:, 0]),1)))

vectorizer_c = TfidfVectorizer(ngram_range = (2, 2))
x_train_c = vectorizer_c.fit_transform(x_train)
x_test_c = vectorizer_c.transform(x_test)

rfc_a = RandomForestClassifier(random_state=random_state, n_jobs=n_jobs, n_estimators=n_estimators)
rfc_b = RandomForestClassifier(random_state=random_state, n_jobs=n_jobs, n_estimators=n_estimators)
rfc_c = RandomForestClassifier(random_state=random_state, n_jobs=n_jobs, n_estimators=n_estimators)

rfc_a.fit(x_train_a, y_train)
rfc_b.fit(x_train_b, y_train)
rfc_c.fit(x_train_c, y_train)

y_pred_a = rfc_a.predict(x_test_a)
y_pred_b = rfc_b.predict(x_test_b)
y_pred_c = rfc_c.predict(x_test_c)

precision_a, recall_a, f1score_a, support_a = precision_recall_fscore_support(y_test, y_pred_a)
precision_b, recall_b, f1score_b, support_b = precision_recall_fscore_support(y_test, y_pred_b)
precision_c, recall_c, f1score_c, support_c = precision_recall_fscore_support(y_test, y_pred_c)

text = pd.read_csv(text_URL, delimiter=";").columns.tolist()
pred = rfc_c.predict(vectorizer_c.transform([text[0]])[0])[0]

print("Ответы:")
print(f"Precision (macro avg) = %.3f" % (sum(precision_a)/len(precision_a)))
print(f"Recall (macro avg) = %.3f" % (sum(recall_a)/len(recall_a)))
print(f"F-score (macro avg) = %.3f" % (sum(f1score_a)/len(f1score_a)))
print()
print(f"Precision (macro avg) = %.3f" % (sum(precision_b)/len(precision_b)))
print(f"Recall (macro avg) = %.3f" % (sum(recall_b)/len(recall_b)))
print(f"F-score (macro avg) = %.3f" % (sum(f1score_b)/len(f1score_b)))
print()
print(f"Precision (macro avg) = %.3f" % (sum(precision_c)/len(precision_c)))
print(f"Recall (macro avg) = %.3f" % (sum(recall_c)/len(recall_c)))
print(f"F-score (macro avg) = %.3f" % (sum(f1score_c)/len(f1score_c)))
print()
print("К классу:", pred)
print(f"Вероятность к классу {pred}", rfc_c.predict_proba(vectorizer_c.transform([text[0]])[0])[0][pred])

Ответы:
Precision (macro avg) = 0.967
Recall (macro avg) = 0.964
F-score (macro avg) = 0.965

Precision (macro avg) = 0.969
Recall (macro avg) = 0.966
F-score (macro avg) = 0.967

Precision (macro avg) = 0.953
Recall (macro avg) = 0.947
F-score (macro avg) = 0.949

К классу: 0
Вероятность к классу 0 0.7
